### Initial Notes on the code objective
This script corresponds to third and last part of the project, corresponding to the Dashboard creation part.

Using the **dash** library it was possible to create an interactive dashboard for data visualisation using the data that was previously treated. 

For the production of the different charts the **plotly** library was used, allowing the user to have an interactive
approach with the different charts, instead of the static approach that many other visualisation libraries that exist in Python provide.

For the design of the dashboard web page it was applied _HTML_ adapted to the Dash library, allowing to create the base structure were all the charts and informations were placed on.

To provide the end-user with an interactive experience two dropdown features were added on the web-page allowing the user to select one or more products to compare from every supermarket that had data on it, and later the capability to choose between each supermarket available for some high-level insights regarding these businesses.


#### Some important for ease visualizing the webpage
 - The webpage is only updated with all the information that is stored on our database if this code is still running
 - After running the whole script the user must click on the link that will appear at the end of this notebook, that will open a new tab on the browser being used.


In [1]:
# install the required packages if needed

#!pip install plotly
#!pip install dash

In [2]:
# Import the required packages

import numpy as np
import pandas as pd

import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output  # pip install dash (version 2.0.0 or higher)

import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [3]:
# Create the dashboard app
app = Dash(__name__)

In [4]:
# Import the data that was previously transformed

df = pd.read_excel("Data_ForDashboard.xlsx")
df = df.sort_values(by = ["Product Object", "Supermarket"])
#df = df.applymap(lambda row: row.lower() if type(row) == str else row)
df.head()

,Product Object,Supermarket,Category Object,Average Per Product,Min Price per Product,Max Price per Product,Count of Products
0,Abóbora,Auchan,Legumes,3.0350,1.69,3.58,4
2,Abóbora,Continente,Legumes,4.0425,1.29,13.27,8
1,Abóbora,Mini Preço,Legumes,3.2000,3.20,3.20,1
4,Agua Garrafão,Auchan,Bebidas,0.2600,0.20,0.30,6
3,Agua Garrafão,Continente,Bebidas,0.1650,0.13,0.20,2


In [5]:
# Definining the supermarket colours for laater use

colours_dict = {'Mini Preço':'yellow',
                'Continente':'red',
                'Auchan':'royalblue'}

In [6]:
# Defining some variables for recurrent use on the dashboard production

## Html Formatting
headers_Dropdown_size = '15px'
headers_Dropdown_font = 'Noto Sans'


background_color = '#E6ECF1'

In [7]:
# --------------------------
# App Layout 

# As of now we are only creating the App layout nothing more,
#    therfore if we only have this code we cannot connect to an interactive server

app.layout = html.Div(style = {'backgroundColor': '#FFFFFF',
                               'margin': '25px'}, # Defining the margin of the page
                      children = [
    html.H1("Supermarkets metrics comparison Dashboard", 
            style={'text-align': 'center',
                   'font-weight': 'bold',
                   'font-family' : headers_Dropdown_font
                  }), # Define the title for the webpage

    #H1 here is just a typical header
    html.H2("Use the different dropdown features across this dashboard to navigate through the supermarket products", 
            style={'text-align': 'center',
                   'font-weight': 'normal',
                   'font-family' : headers_Dropdown_font 
                  }),
    
                          
    html.Div( style={'display': 'inline-block'}, 
             children =[html.Div(
                 style = {'backgroundColor': '#7F90EB', 'width': '300px','height': '75px', 'display': 'inline-block',
                         'margin': 'auto'},
                 children = [html.Div(id='container_Auchan', children=[], 
                                      style={'display': 'inline-block',
                                            'text-align': 'center', 
                                             'font-style': 'italic',
                                             'font-family' : headers_Dropdown_font,
                                             'font-size': headers_Dropdown_size}), # Belongs to the component Div since it creates a "division"
                            ]
             ),
                        
                        html.Div(
                 style = {'backgroundColor': '#9AA6E3', 'width': '300px', 'height': '75px', 'display': 'inline-block',
                          'text-align': 'center', 'font-weight': 'normal','font-family' : headers_Dropdown_font,
                         'margin': 'auto'},                            
                  children = [html.Div(id='container_Continente', children=[], 
                                       style={'display': 'inline-block',
                                              'text-align': 'center', 
                                              'font-style': 'italic',
                                              'font-family' : headers_Dropdown_font,
                                              'font-size': headers_Dropdown_size}), # Belongs to the component Div since it creates a "division
                             ]
                        ),
                        
                        html.Div(
                 style = {'backgroundColor': '#8F96BE', 'width': '300px', 'height': '75px', 'display': 'inline-block',
                         'text-align': 'center', 'font-weight': 'normal','font-family' : headers_Dropdown_font,
                         'margin': 'auto'},
                  children = [html.Div(id='container_MiniPreco', children=[], 
                                       style={'display': 'inline-block',
                                              'text-align': 'center', 
                                              'font-style': 'italic',
                                              'font-family' : headers_Dropdown_font,
                                              'font-size': headers_Dropdown_size}), # Belongs to the component Div since it creates a "division
                            ]                            
                        ),
        
    ]),
        
                          
    # Place an header with the last date that the data was updated 
    html.H3("Last updated as at May 21st 2022", 
            style={'text-align': 'right',
                   'font-style': 'italic',
                   'font-weight': 'bold',
                   'font-family' : headers_Dropdown_font,
                   'font-size': headers_Dropdown_size}),    
    
    html.H4("Select the product or products that you want to search by. Please use the dropdown below:", 
            style={'text-align': 'left',
                   'font-style': 'italic',
                   'font-weight': 'normal',
                   'font-family' : headers_Dropdown_font,
                   'font-size': headers_Dropdown_size}),
    
    dcc.Dropdown(id="slct_product", # to be used later on the component_id parameter
                 options=[
                     {"label": "Abóbora", "value": "Abóbora"},
                     {"label": "Açúcar", "value": "Açúcar"},
                     {"label": "Água Garafão", "value": "Agua Garrafão"},
                     {"label": "Alho", "value": "Alho"},
                     {"label": "Alho Francês", "value": "Alho Frances"},
                     {"label": "Arroz", "value": "Arroz"},
                     {"label": "Azeite", "value": "Azeite"},
                     {"label": "Bacalhau", "value": "Bacalhau"},
                     {"label": "Bananas", "value": "Bananas"},
                     {"label": "Batata", "value": "Batata"},
                     {"label": "Bróculos", "value": "Bróculos"},
                     {"label": "Café", "value": "Cafe"},
                     {"label": "Cebola", "value": "Cebola"},
                     {"label": "Cenoura", "value": "Cenoura"},
                     {"label": "Cereais", "value": "Cereais"},
                     {"label": "Champô", "value": "Champô"},
                     {"label": "Couve-Coração", "value": "Couve Coração"},
                     {"label": "Desodorisante", "value": "desodorisante"},
                     {"label": "Detergente para Chão", "value": "Detergente chao"},
                     {"label": "Detergente para Loica", "value": "Detergente Loiça"},
                     {"label": "Detergente para WC", "value": "Detergente WC"},
                     {"label": "Douradinhos", "value": "Douradinhos"},
                     {"label": "Ervilhas", "value": "Ervilhas"},
                     {"label": "Escova de Dentes", "value": "Escova de Dentes"},
                     {"label": "Esparguete", "value": "Esparguete"},
                     {"label": "Farinha", "value": "Farinha"},
                     {"label": "Feijão", "value": "Feijao"},
                     {"label": "Fiambre", "value": "Fiambre"},
                     {"label": "Frango", "value": "Frango"},
                     {"label": "Grão", "value": "Grão"},
                     {"label": "Iogurtes", "value": "Iogurtes"},
                     {"label": "Laranja", "value": "Laranja"},
                     {"label": "Leite", "value": "Leite"},
                     {"label": "Maçã", "value": "Maçã"},
                     {"label": "Manteiga", "value": "Manteiga"},
                     {"label": "Nabo", "value": "Nabo"},
                     {"label": "Óleo Alimentar", "value": "Oleo Alimentar"},
                     {"label": "Ovos", "value": "Ovos"},
                     {"label": "Pão", "value": "Pão"},
                     {"label": "Papel Higiénico", "value": "Papel higienico"},
                     {"label": "Pasta de Dentes", "value": "Pasta de Dentes"},
                     {"label": "Pêra", "value": "Pera"},
                     {"label": "Perú", "value": "Perú"},
                     {"label": "Porco", "value": "Porco"},
                     {"label": "Queijo", "value": "Queijo"},
                     {"label": "Sabonete", "value": "Sabonete"},
                     {"label": "Sacos do Lixo", "value": "Sacos do Lixo"},
                     {"label": "Salmão", "value": "Salmao"},
                     {"label": "Vaca", "value": "Vaca"},
                     {"label": "Vinagre", "value": "Vinagre"}],                 
                 multi=True, # Here is defined as if we can select only one or multiple items
                 value=['Abóbora'], # This is the initial value of the dropdown
                 style={'width': "40%"}
                 ),

    #    Here the children can be deleted since it is standard but is needed to understand what are its implications
    
    html.Br(), # Create a breakline here to have a space between the div and the graph

    #dcc.Graph(id='my_bee_map', figure={}) # Belongs to the component Grpah since it will plot a chart
    #    Here the figure can be deleted since it is standard but is needed to understand what are its implications
    
    #dcc.Graph(id='Basket_Comparison_Bar', figure={'layout' : {
    #    'title': 'Gráfico simples'
        
    #}}), # Belongs to the component Grpah since it will plot a chart

    
    # Both these charts are referring to the same type of products, therefore they should be next to each other
    # We should place them horizontally distributed
    html.Div([
        dcc.Graph(id='Basket_Comparison_Bubble', style={'display': 'inline-block'}), # Belongs to the component Grpah since it will plot a chart

        dcc.Graph(id='BasketProduct_Comparison_Bubble', style={'display': 'inline-block'}), # Belongs to the component Grpah since it will plot a chart
        
        dcc.Graph(id='PieChart_SelectProducts', style={'display': 'inline-block'}), # Belongs to the component Grpah since it will plot a chart

    ]),#, style={'display': 'inline-block', 'vertical-align': 'top'}),
            
    
    html.H5("Select the Supermarket that you want to search by. Please use the dropdown below:", 
            style={'text-align': 'left',
                   'font-style': 'italic',
                   'font-weight': 'normal',
                   'font-family' : headers_Dropdown_font,
                   'font-size': headers_Dropdown_size}),
    
    dcc.Dropdown(id="slct_SuperMarket", # to be used later on the component_id parameter
                 options=[
                     {"label": "Auchan", "value": "Auchan"},
                     {"label": "Continente", "value": "Continente"},
                     {"label": "Mini-Preço", "value": "Mini Preço"}],
                 multi=False, # Here is defined as if we can select only one or multiple items
                 value='Auchan', # This is the initial value of the dropdown
                 style={'width': "40%"}
                 ),
    html.Br(), # Create a breakline here to have a space between the div and the graph



    html.Div([
        dcc.Graph(
            id='Supermarket_AllProducts_Comparison', figure={}
        ), # Belongs to the component Grpah since it will plot a chart
    ], style={'width': '100%','height': '500px','display': 'inline-block', 'vertical-align': 'middle'}),
    
    html.Br(), # Create a breakline here to have a space between the div and the graph  
    
    # Categories comparison
    html.Br(), # Create a breakline here to have a space between the div and the graph
    html.Div([
        dcc.Graph(id='Categories_AllProducts_Comparison_bar', style={'display': 'inline-block'}), # Belongs to the component Grpah since it will plot a chart

        dcc.Graph(id='Categories_AllProducts_Comparison_scatter', style={'display': 'inline-block'}), # Belongs to the component Grpah since it will plot a chart
    ]),
    html.Div([
        dcc.Graph(id='Categories_PieChart', style={'display': 'inline-block'}), # Belongs to the component Grpah since it will plot a chart
        dcc.Graph(id='Categories_BarChart', style={'display': 'inline-block'}), # Belongs to the component Grpah since it will plot a chart
    ]),
    
])


In [8]:
df.head()

,Product Object,Supermarket,Category Object,Average Per Product,Min Price per Product,Max Price per Product,Count of Products
0,Abóbora,Auchan,Legumes,3.0350,1.69,3.58,4
2,Abóbora,Continente,Legumes,4.0425,1.29,13.27,8
1,Abóbora,Mini Preço,Legumes,3.2000,3.20,3.20,1
4,Agua Garrafão,Auchan,Bebidas,0.2600,0.20,0.30,6
3,Agua Garrafão,Continente,Bebidas,0.1650,0.13,0.20,2


In [9]:
basket_price = df.copy()
basket_price['Product Exists'] = df.apply(lambda row : 1 if row['Count of Products'] else 0, axis=1) # Defining a counter for the products that were retrieved

basket_price = basket_price.groupby('Supermarket').sum().sort_values(by = 'Average Per Product')

basket_price


container_Auchan = basket_price[basket_price.index == "Auchan"]
container_Auchan = str(round(container_Auchan.iloc[0,0]/container_Auchan.iloc[0,4],2))
container_Auchan = 'The average product price for Auchan is  {}'.format(
        container_Auchan + str( "€ per product")) 

container_Auchan

'The average product price for Auchan is  4.6€ per product'

### Prepare the charts to be deployed using the callback and the update_graph function

In [10]:
@app.callback(
    [#Output(component_id='output_container', component_property='children'), # Is going to output a children of the output_container (defined previously)
        
     # DEfinition of the KPI boxes at the beginning of the dashboard
     Output(component_id='container_Auchan', component_property='children'),
     Output(component_id='container_Continente', component_property='children'),
     Output(component_id='container_MiniPreco', component_property='children'),
     
     #Output(component_id='Basket_Comparison_Bar', component_property='figure'),
     Output(component_id='Basket_Comparison_Bubble', component_property='figure'),
     Output(component_id='BasketProduct_Comparison_Bubble', component_property='figure'),
        
     # Pie Chart
     Output(component_id='PieChart_SelectProducts', component_property='figure'),
        
        
     Output(component_id='Supermarket_AllProducts_Comparison', component_property='figure'), # This plots the data on the chart, and is going to be a figure (as defined previously)
     
     
     Output(component_id='Categories_AllProducts_Comparison_bar', component_property='figure'), # This plots the data on the chart, and is going to be a figure (as defined previously)
     Output(component_id='Categories_AllProducts_Comparison_scatter', component_property='figure'), # This plots the data on the chart, and is going to be a figure (as defined previously)

     Output(component_id='Categories_PieChart', component_property='figure'), # This plots the data on the chart, and is going to be a figure (as defined previously)
     Output(component_id='Categories_BarChart', component_property='figure') # This plots the data on the chart, and is going to be a figure (as defined previously)

    
    ], # This plots the data on the chart, and is going to be a figure (as defined previously)
    
    
    # Now we are defining the inputs, that will be both dropdowns
    [Input(component_id='slct_product', component_property='value'),
     Input(component_id='slct_SuperMarket', component_property='value')
    
    ]

)

# It will use the component_ID and the component_property to connect both figures
    # The component_id is defined as the slct_year

# Every callback has an output and has an input
# On this scenario we have two outputs and only one input because we will be plotting info in both the output container
#    and on the my_bee_app


# Since our chart is expected to be interactive then we need to create a function that updates the chart 
    # whenever we change the value in the dropdown list (option_slctd)
def update_graph(option_slctd, slct_SuperMarket):
    
    # 1. Average product price for a supermarket
    average_price = df.groupby('Supermarket').sum().sort_values(by = 'Average Per Product')
    average_price['Average Per Product'] = average_price['Average Per Product'] /50


    container_initial = "The average price for {} is {}".format(slct_SuperMarket,
                                                                str(round(average_price[average_price.index == slct_SuperMarket]["Average Per Product"][0],2)) + str( "€ per product"))   
    
    
    basket_price = df.copy()
    basket_price['Product Exists'] = df.apply(lambda row : 1 if row['Count of Products'] else 0, axis=1) # Defining a counter for the products that were retrieved
    basket_price = basket_price.groupby('Supermarket').sum().sort_values(by = 'Average Per Product')
    
    
    container_Auchan = basket_price[basket_price.index == "Auchan"]
    container_Auchan = str(round(container_Auchan.iloc[0,0]/container_Auchan.iloc[0,4],2))
    container_Auchan = 'The average product price for Auchan is  {}'.format(
        container_Auchan + str( "€ per product")) 

    container_Continente = basket_price[basket_price.index == "Continente"]
    container_Continente = str(round(container_Continente.iloc[0,0]/container_Continente.iloc[0,4],2))
    container_Continente = 'The average product price for Continente is  {}'.format(
        container_Continente + str( "€ per product")) 

    container_MiniPreco = basket_price[basket_price.index == "Mini Preço"]
    container_MiniPreco = str(round(container_MiniPreco.iloc[0,0]/container_MiniPreco.iloc[0,4],2))
    container_MiniPreco = 'The average product price for Mini Preço is  {}'.format(
        container_MiniPreco + str( "€ per product")) 

    
    
    


    # 3. 
    container_lastUpd = 'Last updated as at May 22nd 2022'
    
    ### ------------ beginning of the charts definition ------------------- ###

    ### Chart 1. Overall Basket Price comparison using a Bar Chart in horizontal orientation
    
    basket_price = df.groupby('Supermarket').sum().sort_values(by = 'Average Per Product')

    fig_1 = go.Figure(go.Bar(
        x=basket_price["Average Per Product"], # Considering the aggregated price
        y=basket_price.index, # using the index that has the names of all supermarkets
        orientation='h',
        text=basket_price["Average Per Product"], # Defining the value that will be placed
        textposition='auto' # Defining the orientation of the chart
    ))

    fig_1.update_layout(
        autosize=False,
        width=1000,
        height=400,)
    
    fig_1.update_layout(paper_bgcolor=background_color, plot_bgcolor = background_color)
    
    
    
    
    ### Chart 2. Overall basket Price comparison using Bubble Chart
    
    # Get the overall basket price for each SuperMarket
    #basket_price = df.groupby('SuperMarket').sum().sort_values(by = 'Price')

    basket_price = df.copy()
    basket_price['Product Exists'] = df.apply(lambda row : 1 if row['Count of Products'] else 0, axis=1) # Defining a counter for the products that were retrieved

    basket_price = basket_price.groupby('Supermarket').sum().sort_values(by = 'Average Per Product')

    fig_2 = px.scatter(x=basket_price['Average Per Product'], y=basket_price["Product Exists"],
                     size=basket_price["Average Per Product"], # Here we could place the number of products that are available 
                     color=basket_price.index,
                     color_discrete_map={
                         basket_price.index[0]: colours_dict[basket_price.index[0]],
                         basket_price.index[1]: colours_dict[basket_price.index[1]],
                         basket_price.index[2]: colours_dict[basket_price.index[2]]
                     },
                     hover_name=basket_price["Average Per Product"], size_max=60,
                    text=basket_price["Product Exists"], # Defining the value that will be placed
                    #textposition='auto'
                    )
    # Remove x axis
    fig_2.update_xaxes(visible=False, showticklabels=False)

    # Add legend
    fig_2.update_layout(showlegend=True)
    fig_2.update_layout(showlegend=True,
                        paper_bgcolor=background_color, 
                        plot_bgcolor = background_color,
                        title_text='Fixed Basket Price for each supermarket',
                        xaxis=dict(showgrid=False),
                        yaxis=dict(showgrid=False),
                        title_x=0.5
                       )
    fig_2.update_layout(
        yaxis_title="Number of products count"
    )
    
    ### Chart 3. Product Price comparison letting user decide which products to choose
    
    """ 
    Chart to be defined based on multiple criteria selection
    """
    basket_price_products = df[df["Product Object"].isin(option_slctd)].groupby('Supermarket').sum()
    
    fig_3 = px.scatter(x=basket_price_products['Average Per Product'], y=basket_price_products["Count of Products"],
                     size=basket_price_products["Average Per Product"], # Here we could place the number of products that are available 
                     color=basket_price_products.index,
                     color_discrete_map={
                     basket_price_products.index[0]: colours_dict[basket_price_products.index[0]],
                     basket_price_products.index[1]: colours_dict[basket_price_products.index[1]],
                     basket_price_products.index[2]: colours_dict[basket_price_products.index[2]]   
                 },
                     hover_name=basket_price_products["Average Per Product"], size_max=60,
                )

    # Add legend
    fig_3.update_layout(showlegend=True)

    # Update the figure size and title
    fig_3.update_layout(title_text='Basket Price for the select products for each supermarket',
                        paper_bgcolor=background_color, 
                        plot_bgcolor = background_color,
                        xaxis=dict(showgrid=False),
                        yaxis=dict(showgrid=False),
                        title_x=0.5)


    # Remove grid
    fig_3.update_layout(xaxis=dict(showgrid=False),
                        yaxis=dict(showgrid=False)
                       )

    # Remove x axis
    fig_3.update_xaxes(visible=False, showticklabels=False)

    # Add label to y axis
    fig_3.update_layout(
        yaxis_title="Number of products count"
    )
    
    
    
    ### Chart 4. All prices from a supermarket

    supermarket = slct_SuperMarket # Defining the super market
    
    #supermarket = 'mini preço'

    categories_prices = df[df["Supermarket"] == supermarket].sort_values(by = 'Average Per Product', ascending = False)

    fig_4 = px.scatter(categories_prices, x="Product Object", 
                       y=[categories_prices["Min Price per Product"],
                          categories_prices["Average Per Product"], 
                          categories_prices["Max Price per Product"]], 
                       title="Products' Price Range with product count comparison",
                       color_discrete_map={
                           'Min Price per Product': '#1A2BCE',
                           'Average Per Product': '#14A511',
                           'Max Price per Product': '#E90B0B'},
                       size = 'Count of Products'
                      )
    fig_4.update_traces(textposition="bottom right")

    fig_4.update_layout(paper_bgcolor=background_color, 
                      plot_bgcolor = background_color,
                      #width = 1200,
                      xaxis=dict(showgrid=False),
                      yaxis=dict(showgrid=False),
                      title_x=0.5)
    fig_4.update_layout(
        yaxis_title="Price (€/Kg or €/L)"
    )
    
    ### ----- CATEGORY COMPARISON ------- ###
    ### Chart 5. Price comparison based on products' category (Bar Chart)
    
    categories_prices = df[df["Supermarket"] == slct_SuperMarket]
    categories_prices = categories_prices.groupby('Category Object').sum().sort_values(by = 'Average Per Product')
    categories_prices["Category Object"] = categories_prices.index

    fig_5 = px.bar(categories_prices, x="Category Object", 
                   y=[categories_prices["Min Price per Product"],
                      categories_prices["Average Per Product"], 
                      categories_prices["Max Price per Product"]], 
                 title="Category's Price Range", 
                 color_discrete_map={
                         'Min Price per Product': '#1A2BCE',
                         'Average Per Product': '#14A511',
                         'Max Price per Product': '#E90B0B'}
                 )
    
    fig_5.update_layout(
        yaxis_title="Price (€/Kg or €/L)")
    
    
    fig_5.update_layout(paper_bgcolor=background_color, plot_bgcolor = background_color,
                       title_x=0.5)
    
    ### Chart 6. Price comparison based on products' category (Scatter)

    categories_prices = df[df["Supermarket"] == slct_SuperMarket]
    categories_prices = categories_prices.groupby('Category Object').sum().sort_values(by = 'Average Per Product')
    categories_prices["Category Object"] = categories_prices.index

    fig_6 = px.scatter(categories_prices, x="Category Object", 
                       y=[categories_prices["Min Price per Product"],
                          categories_prices["Average Per Product"], 
                          categories_prices["Max Price per Product"]], 
                     title="Category's Price Range with product count comparison", 
                     color_discrete_map={
                         'Min Price per Product': '#1A2BCE',
                         'Average Per Product': '#14A511',
                         'Max Price per Product': '#E90B0B'}, 
                     size = 'Count of Products'
                    )
    fig_6.update_layout(
        yaxis_title="Price (€/Kg or €/L)",
        title_x=0.5
        
    )
    fig_6.update_layout(paper_bgcolor=background_color, plot_bgcolor = background_color)  
    
    
    
    #figure 7
    basket_price = df.copy()
    basket_price_categories = basket_price[basket_price["Supermarket"] == slct_SuperMarket]

    basket_price_categories = basket_price_categories.groupby('Category Object').sum().sort_values(by = 'Count of Products')
    basket_price_categories['Product Exists'] = basket_price_categories.apply(lambda row : 1 if row['Count of Products'] else 0, axis=1) # Defining a counter for the products that were retrieved



    fig_7 = px.pie(basket_price_categories, values='Count of Products', names=basket_price_categories.index, 
             title=str('Percentage of the total products available that exist in '+str(slct_SuperMarket)),

            )
    fig_7.update_traces(textposition='inside', textinfo='percent+label')

    fig_7.update_layout(showlegend=True,
                        paper_bgcolor=background_color, 
                        plot_bgcolor = background_color,
                      title_x=0.5)
    
    # Figure 8
    fig_8 = go.Figure(go.Bar(
    y = basket_price_categories.index,
    x  = basket_price_categories['Count of Products'],
            orientation='h'))


    fig_8.update_layout(paper_bgcolor=background_color, 
                      plot_bgcolor = background_color,
                      title = str("Available products from the white brand divied by category for "+ str(supermarket)),
                      title_x=0.5)
    
    # Figure 9
    basket_price = df.copy()
    basket_price['Product Exists'] = df.apply(lambda row : 1 if row['Count of Products'] else 0, axis=1) # Defining a counter for the products that were retrieved


    basket_price_products = basket_price.groupby('Supermarket').sum()
    basket_price_products


    fig_9 = px.pie(basket_price_products, values='Count of Products', names=basket_price_products.index, 
             title='Percentage of the total products available that exist in each Supermarket',
            color = basket_price_products.index,
             color_discrete_map={
                basket_price_products.index[0]: colours_dict[basket_price_products.index[0]],
                 basket_price_products.index[1]: colours_dict[basket_price_products.index[1]],
                 basket_price_products.index[2]: colours_dict[basket_price_products.index[2]]
                     }
            )
    fig_9.update_traces(textposition='inside', textinfo='percent+label')

    fig_9.update_layout(showlegend=True,
                        paper_bgcolor=background_color, 
                        plot_bgcolor = background_color,
                        title_x=0.5)    
    
    
    

    return container_Auchan, container_Continente, container_MiniPreco, fig_2, fig_3, fig_9, fig_4, fig_6, fig_5,fig_7, fig_8

### Deploy the dashboard
 - Please use the link that will appear at the end of the notebook to open and interact with the dashboard

In [11]:
# For Jupyter    
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
